In [14]:
!pip install pyTelegramBotAPI
!pip install ipynb
!pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [15]:
!pip install efficientnet_pytorch
!pip install opencv-python
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

"$(pip freeze | grep albumentations) is successfully installed"


In [16]:
import telebot
import torch
from efficientnet_pytorch import EfficientNet
import requests
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import numpy as np

In [17]:
model2 = torch.load('D:\\model_v2',map_location=torch.device('cpu'))   #D:\\model

In [18]:
def get_image(img):
          x = cv2.imread(img)
          x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
          transform2 = A.Compose([
                                 A.Resize(256,256),
                                 A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                                 ToTensorV2(),
          ])
          if transform2 is not None:
             x = transform2(image=x)['image']
          return x

In [19]:
def predict(model, img):
    with torch.no_grad():
        results = []
        x = get_image(img).unsqueeze(0)
        #x = x.to(torch.device("cuda"))
        model.eval()
        res = model(x)
        probability = nn.functional.softmax(res, dim=-1).numpy()
    return probability

In [20]:
def visualize(image):
    x = cv2.imread(image)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(x)

In [21]:
file = open(r'C:\\Users\79192\Desktop\project2\last_version\TOKEN2.txt', 'r')
API_TOKEN= file.read()
file.close()

In [22]:
bot = telebot.TeleBot(API_TOKEN)

In [23]:
@bot.message_handler(commands=['start'])
def welcome(message):    
  bot.send_message(message.chat.id, 'Привет! Отправь мне фото материала, а я скажу тебе имеет он дефект или нет.')

In [24]:
@bot.message_handler(content_types=["photo","document"])
def answer(message): 
    bot.send_message(message.chat.id, 'Проверяю...')
    if message.photo is not None:
        file_info = bot.get_file(message.photo[-1].file_id)
    elif message.document is not None:
        file_info = bot.get_file(message.document.file_id)
    else: 
        bot.send_message(message.chat.id, 'Ваше сообщение не содержит файлов и фотографий. Пожалуйста отправте фотографию детали')
    img = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
    img_file = open('img_file', 'wb')
    img_file.write(img.content)
    img_file.close()
    prob = predict(model2,'img_file')
    result = int(np.argmax(prob,-1))
    prob = np.reshape(np.array(prob), (20,))
    for i in range(0,10):
      if (result == i):
          bot.send_message(message.chat.id, 'С вероятностью ' + str(prob[i]) + ' это деталь класса ' + str(i) + ' без дефектов')
    for i in range(10,20):
      if (result == i):
          bot.send_message(message.chat.id, 'С вероятностью ' + str(prob[i]) + ' это деталь класса ' + str(i-10) + ' c дефектом')
    bot.send_message(message.chat.id, 'Отправте новую фотографию детали, если хотите узнать имеет она дефекты или нет')

In [ ]:
while True:    
  try:        
    bot.polling(none_stop=True)    
  except ():        
    time.sleep(5)